In [1]:
import numpy as np
import os
from tqdm.auto import tqdm
# from pathos.multiprocessing import ProcessingPool as Pool
# from functools import partial
import pandas as pd
from PIL import Image

In [2]:
# clone Scaled_YOLOv4
!git clone https://github.com/tom-auger/ScaledYOLOv4.git
%cd ScaledYOLOv4/
#checkout the yolov4-large branch
!git checkout yolov4-large

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 156 (delta 18), reused 19 (delta 9), pack-reused 128
Receiving objects: 100% (156/156), 355.12 KiB | 4.38 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/kaggle/working/ScaledYOLOv4
Branch 'yolov4-large' set up to track remote branch 'yolov4-large' from 'origin'.
Switched to a new branch 'yolov4-large'


In [3]:
df_train = pd.read_csv('/kaggle/input/mlpyoloall/train.csv')
df_val = pd.read_csv('/kaggle/input/mlpyoloall/validation.csv')
df_test = pd.read_csv('/kaggle/input/mlpyoloall/test.csv')

In [4]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.7.0 CPU


In [5]:
#install mish activation funciton for cuda
%cd ..
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

/kaggle/working
Cloning into 'mish-cuda'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 2.75 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/kaggle/working/mish-cuda
Traceback (most recent call last):
  File "setup.py", line 26, in <module>
    'nvcc': ['--expt-extended-lambda']
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 783, in CUDAExtension
    library_dirs += library_paths(cuda=True)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 875, in library_paths
    if (not os.path.exists(_join_cuda_home(lib_dir)) and
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1794, in _join_cuda_home
    raise EnvironmentError('CUDA_HOME environment variable is not set. '
OSError: CUDA_HOME 

In [6]:
abnormalities = ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']

%mkdir data
def write_to_file(imgs, dataset_name, ab_name):
    with open(f"data/{dataset_name}_{ab_name}.txt", 'wt') as f:
        for img in imgs:
            f.write(f"/kaggle/input/mlpyoloall/{dataset_name}/{img}.png\n")
    
def write_yaml_file(ab):
    with open(f"data/{ab}.yaml", 'wt') as f:
        f.write(f"""train: ../data/train_{ab}.txt
val: ../data/validation_{ab}.txt

nc: 14
names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
""")
        
for ab in abnormalities:
    train_imgs = df_train[df_train['class_name'] == ab]['image_id'].unique()
    val_imgs = df_val[df_val['class_name'] == ab]['image_id'].unique()
    test_imgs = df_test[df_test['class_name'] == ab]['image_id'].unique()
    
    # Write each list to a data spec file
    ab = ab.replace('/', '_').replace(' ', '_')
    write_to_file(train_imgs, 'train', ab)
    write_to_file(val_imgs, 'validation', ab)
    write_to_file(test_imgs, 'test', ab)
    
    # Write the data yaml file
    write_yaml_file(ab)

In [7]:
%cd ScaledYOLOv4/

/kaggle/working/ScaledYOLOv4


In [8]:
!python train.py --img 640 --batch 16 --epochs 100 --data '../data/Calcification.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results  --cache

Traceback (most recent call last):
  File "train.py", line 20, in <module>
    import test  # import test.py to get mAP after each epoch
  File "/kaggle/working/ScaledYOLOv4/test.py", line 13, in <module>
    from models.experimental import attempt_load
  File "/kaggle/working/ScaledYOLOv4/models/experimental.py", line 7, in <module>
    from models.common import Conv, DWConv
  File "/kaggle/working/ScaledYOLOv4/models/common.py", line 7, in <module>
    from mish_cuda import MishCuda as Mish
ModuleNotFoundError: No module named 'mish_cuda'
